<a href="https://colab.research.google.com/github/puneet5056/Java/blob/master/June18_WD_Morning_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
Hi welcome

CREATE EXTERNAL TABLE `emp_csv`(
  `id` string COMMENT 'from deserializer',
  `name` string COMMENT 'from deserializer',
  `dept` string COMMENT 'from deserializer',
  `joindate` string COMMENT 'from deserializer',
  `salary` string COMMENT 'from deserializer')
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'escapeChar'='\\',
  'quoteChar'="\"",
  'separatorChar'=',')
STORED AS INPUTFORMAT
  'org.apache.hadoop.mapred.TextInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://localhost:9000/hive/practice/emp'
TBLPROPERTIES (
  'transient_lastDdlTime'='1536548978')


1,stu1,dept1,2010-01-01,1000
2,stu2,dept1,2010-01-01,2000
3,stu3,dept1,2011-01-01,3000
4,stu4,dept2,2011-01-01,500
5,stu5,dept2,2012-01-01,500
6,stu6,dept2,2012-01-01,500
7,stu7,"de,pt3",2012-01-01,2000
8,stu8,"de,pt3",2012-01-01,2000


load data local inpath 'empdata' into table emp_csv;
load data local inpath 'empdata' into table emp_csv;

In [0]:
Hive Partitioning
emp_csv as source 
Create a partitioned tble emp_part - good candidates for partitioning column are
dept, year, month or fiscal week

on emp id we can go for bucketing 
hive (hivepractice)> desc emp_csv
hive (hivepractice)> select * from emp_csv;
hive (hivepractice)> load data local inpath 'empdata' into table emp_csv;
Time taken: 0.102 seconds, Fetched: 16 row(s)


In [0]:
create table emp_part with concat of year and month
Create with delimiter as # field delimiter
hive (hivepractice)> show create table emp_csv;
CREATE EXTERNAL TABLE `emp_part`(
  `id` string ,
  `name` string ,
  `dept` string ,
  `joindate` string ,
  `salary` string )
partitioned by (fiscalmonth int)
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'escapeChar'='\\',
  'quoteChar'="\"",
  'separatorChar'='#')
stored as textfile;

hive (hivepractice)> show create table emp_part;
desc  emp_part;
# Partition Information
# col_name              data_type               comment

fiscalmonth             int

hive (hivepractice)> show partitions emp_part;

hive (hivepractice)> dfs -ls /hive/practice/emp_part;


 select *, concat(year(joindate),month(joindate)) from emp_csv;

   select if(month("2016-11-20")<10,
             concat("0",month("2016-11-20")),
             month("2016-11-20")) ;

     select concat(year(joindate),                    
             if(month(joindate)<10,
             concat("0",month(joindate)),
             month(joindate))
                  )  from emp_csv;

      
 insert into table emp_part  partition(fiscalmonth) 
select * , 
concat(year(joindate), 
 if(month(joindate)<10,
    concat("0",month(joindate)),
    month(joindate)
   )
  )  from emp_csv;
set hive.exec.dynamic.partition.mode=nonstrict;
----------------------------
--  insert into table emp_part  partition(fiscalmonth=201001) 
--  LOAD DATA LOCAL  INPATH 'emp1'  
INTO TABLE emp_part PARTITION (fiscalmonth=201001)
1,stu1,dept1,2010-01-01,1000
2,stu2,dept1,2010-01-01,2000
----------------------------

-- insert overwrite affects only the partitions that are present in select query.
-- other partitions are uneffected.

insert overwrite  table emp_part  partition(fiscalmonth) 
select * , 
concat(year(joindate), 
 if(month(joindate)<10,
    concat("0",month(joindate)),
    month(joindate)
   )
  )  from emp_csv where concat(year(joindate), 
 if(month(joindate)<10,
    concat("0",month(joindate)),
    month(joindate)
   )
  )   =201001;
-------------------------------------------------

hive (hivepractice)> truncate table emp_part;
FAILED: SemanticException [Error 10146]: Cannot truncate non-managed table emp_part.

hive (hivepractice)> dfs -mkdir -p /hive/practice/emp_part/fiscalmonth=9999;
hive (hivepractice)> msck repair table emp_part;
hive (hivepractice)> alter table emp_part set TBLPROPERTIES ("EXTERNAL"="FALSE")

Partitions not in metastore:    emp_part:fiscalmonth=9999
Partitions missing from filesystem:     emp_part:fiscalmonth=201001     emp_part:fiscalmonth=201101     emp_part:fiscalmonth=201201
Repair: Added partition to metastore emp_part:fiscalmonth=9999

    MSCK does not remove metadata, it only add metadata (if we created hdfs folder manullay)
    


In [0]:
Bucketing ::
    
    divide table data into fixed no of buckets (Files)
    Hash partition
    
      [CLUSTERED BY (col_name, col_name, ...) [SORTED BY (col_name [ASC|DESC], ...)] INTO num_buckets BUCKETS]
      
      CREATE EXTERNAL TABLE `emp_bucketed`(
  `id` string ,
  `name` string ,
  `dept` string ,
  `joindate` string ,
  `salary` string )
      
  clustered by (id) into 3 buckets
 ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'escapeChar'='\\',
  'quoteChar'="\"",
  'separatorChar'='#')
   
      stored as textfile;

    desc formatted emp_bucketed;
    Num Buckets:            3
    Bucket Columns:         [id]

    
    DML ???  - it is always insert overwrite in to bucketed table from non bucketed table.
    
    Location:               hdfs://localhost:9000/hive/practice/emp_bucketed

    insert overwrite table emp_bucketed select * from emp_csv;
    
    
    
     CREATE EXTERNAL TABLE `emp_bucketed_sorted`(
  `id` string ,
  `name` string ,
  `dept` string ,
  `joindate` string ,
  `salary` string )
      
  clustered by (id) sorted by (id) into 3 buckets
 ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.OpenCSVSerde'
WITH SERDEPROPERTIES (
  'escapeChar'='\\',
  'quoteChar'="\"",
  'separatorChar'='#') ;


    insert overwrite table emp_bucketed_sorted select * from emp_csv;


In [0]:
Clauses in Select query
Select distinct, *, col1, col2 , functions (col) - UDF, UDAF, UDTF, windown and analytic funcitons
from A join B on A.col1=B.col2 
join (select * from C) C1 on A.col2=C1.col3
where A.col1 > 10  and  ( UDF(b.col2)=100 or C1.col3=15)
group by A.col3 -- the columns not in group by clause have to be used with UDAFs in select query
having UDAF(col4)>3  count(dept)>3 -- we can filter rows based on the result of aggregate funcitons in the same query
orderby -- dont use -> global aggregation -> launch one reducer
limit 10

Actual execution is -> from, where, group by , having , limit , order by , select 
windowing and analytic functions evaluated , so we cannot filter data based on 
analytic funcitons within the same query - store it into table or make it as subquery , then you can apply where conditiin


In [0]:
FUNCTIONS::
    https://cwiki.apache.org/confluence/display/Hive/LanguageManual+UDF

In [0]:
show functions;

In [0]:
Date : 13 Sept 2018
  
MOVIE LENS Analysis


1. Top ten most viewed movies with their movies Name (Ascending or Descending order)
2. Top twenty rated movies (Condition : The movie should be rated/viewed by at least 40 users)
3. Top twenty rated movies (which is calculated in the previous step) with no of views in the
following age group
(Age group : 1. Young (<20 years),
2. Young Adult(20-40 years),
3.adult (> 40years) )
(CROSS tab report we need to generate)
movieid    20   20-40  40
1       cnt      cnt   cnt
2
3

4. Top ten critics (Users who have given very low ratings; Condition : The users should have at least
rated 40 movies)




mkdir $HOME/Movielens_wd_morning
cd  $HOME/Movielens_wd_morning
wget https://github.com/dorababugjntup/Assignments/raw/master/3_MovieLens/ml-1m.zip
sudo apt-get install unzip 

unzip ml-1m.zip



Clean data sets:::
      
create a table with one column -> replace :: with : or #  - movie_datalake.ratings_ip 
create a external table with the columns -> hdpmovielens.ratings
      
  
create database movie_datalake;
create database hdpmovielens;

set srcdb=movie_datalake;
set targetdb=hdpmovielens;

use movie_datalake;
create table ratings_ip( line string)
stored as textfile;

/home/dorababugjntup/Movielens_wd_morning/ml-1m
load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/ratings.dat' into table ${hiveconf:srcdb}.ratings_ip;
--insert into ratings   select split(line,"::")[0],split(line,"::")[1],split(line,"::")[2] from ratings_ip;

select line from ratings_ip limit 10;


insert overwrite directory "/hive/movielens/ratings"  select replace(line,"::",":") from ratings_ip ;
create table hdpmovielens.ratings 
(userid int,
 movieid int ,
 rating int, 
 createdtime bigint)
row format delimited fields terminated by ':'
stored as textfile
location '/hive/movielens/ratings';


Create Movie and User table, 
in movie table i want the movie released year as a column.

load the data



In [0]:
create database movie_datalake;
create database hdpmovielens;
set srcdb=movie_datalake;
set targetdb=hdpmovielens;
use movie_datalake;
create table ratings_ip( line string)
stored as textfile;
truncate table ratings_ip;
select line from ratings_ip limit 10;
load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/ratings.dat' into table ${srcdb}.ratings_ip;
set srcdb;
load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/ratings.dat' into table '${srcdb}'.ratings_ip;
set srcdb;
load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/ratings.dat' into table ${hiveconf:srcdb}.ratings_ip;
select line from ratings_ip limit 10;
select count(1) from ratings_ip;
select line from ratings_ip limit 10;
desc function extended concat
;
SELECT concat('abc', 'def') ;
SELECT concat('abc',"===", 'def') ;
SELECT concat('abc',"===", 'def', "===", null) ;
SELECT concat('abc',"===", 'def', "===", nvl( null, "DEFAULT")) ;
desc function extended concat_ws;
SELECT concat_ws("==", 'abc', 'def',NULL) ;
SELECT concat_ws("==", 'abc', 'def',NULL, "abcdedf" ) ;
SELECT concat( 'abc', 'def',NULL, "abcdedf" ) ;
desc function extended concat_ws;
select split("a b c d e"," ");
select split("a b c d e"," ")[0];
select split("a b c d e"," ")[2];
select array("test","test1","test2")
;
select array("test","test1","test2")[2]
;
select array("test","test1","test2")[3]
;
select array("test","test1","test2")[100000]
;
select map("maths",90,"science",100);
select map_keys(map("maths",90,"science",100));
select map_values(map("maths",90,"science",100));
select map("maths",90,"science",100)["science"];
select map("maths",90,"science",100)["science1"];
select size(array("test","test1","test2"));
show tables;
select replace(line,"::",":") from ratings_ip limit 10;
dfs -ls /hive;
insert overwrite directory "/hive/movielens/ratings"  select replace(line,"::",":") from ratings_ip ;
dfs -cat /hive/movielens/ratings/*;
dfs -ls  /hive/;
dfs -ls  /hive/movielens;
dfs -ls  /hive/movielens/ratings;
select cast (956704519 as int)
;
select cast (956704519999999 as int)
;
select cast (9999999999999 as int)
;
select cast (9999999999999 as bigint);
select cast (9999999999 as int);
select cast (9999999 as int);
select cast (99999999 as int);
select cast (999999999 as int);
select cast (9999999999 as int);
create table hdpmovielens.ratings
(userid int,
 movieid int ,
 rating int,
 timestamp bigint)
row format delimited fields terminated by ':'
stored as textfile
location '/hive/movielens/ratings';
create table hdpmovielens.ratings
(userid int,
 movieid int ,
 rating int,
 time bigint)
row format delimited fields terminated by ':'
stored as textfile
location '/hive/movielens/ratings';
use hdpmovielens;
select id from ratings limit 10;
select userid from ratings limit 10;
select ratings from ratings limit 10;
select rating from ratings limit 10;
select movieid,count(movieid) as cnt from ratings  group by movieid ;
select movieid,count(movieid) as cnt from ratings where movieid=661  group by movieid ;


In [0]:
Ratings, users and movies::
DATE:::    17 sept 2018
     
create table movie_datalake.users_ip( line string)
stored as textfile;

UserID::Gender::Age::Occupation::Zip-code
load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/users.dat' into table movie_datalake.users_ip;

insert overwrite directory "/hive/movielens/user"  select replace(line,"::",":") from movie_datalake.users_ip ;
create table hdpmovielens.users 
(userid int,
 gender string ,
 age int, 
 occupation string,
zipcode int)
row format delimited fields terminated by ':'
stored as textfile
location '/hive/movielens/user';
select * from hdpmovielens.users limit 10;
 
  
  MovieID::Title::Genres
  create table movie_datalake.movies_ip( line string)
stored as textfile;

load data local inpath '${system:user.home}/Movielens_wd_morning/ml-1m/movies.dat' 
into table movie_datalake.movies_ip;

insert overwrite directory "/hive/movielens/movies"  select replace(line,"::","@") 
from movie_datalake.movies_ip ;


  MovieID::Title::Genres
movieid, title , release_year , genres (ARRAY)

create table hdpmovielens.movies_stg
(movieid int,
 title string ,
 genres Array<string>
)
row format delimited fields terminated by '@'
collection items terminated by "|"
stored as textfile
location '/hive/movielens/movies';

hive (movie_datalake)> select title from hdpmovielens.movies_stg;






select substr("3845::And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)::Drama",60,4);
select reverse(substr(reverse("3845::And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)::Drama"),instr(reverse("3845::And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)::Drama"),")")+1,4));


select reverse(substr(reverse("3845::And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)::Drama"),instr(reverse("3845::And God Created Woman (Et Dieu&#8230;Créa la Femme) (1956)::Drama"),")")+1,4));


select title from hdpmovielens.movies_stg;
select reverse(substr(reverse(title),
                      instr(reverse(title),")")+1,4)) from hdpmovielens.movies_stg;


select reverse(substr(reverse(title),
                      instr(reverse(title),")")+1,4)) from hdpmovielens.movies_stg;

create table hdpmovielens.movies
(movieid int,
 title string ,
 release_year int, 
 genres Array<string>
)
row format delimited fields terminated by '@'
collection items terminated by "|"
stored as textfile;

insert overwrite table hdpmovielens.movies select movieid,title,
                      reverse(substr(reverse(title),
                      instr(reverse(title),")")+1,4)) as title_year,
                      genres from hdpmovielens.movies_stg ;

select release_year, movieid from hdpmovielens.movies limit 10;


    

In [0]:
Date 18 Sept 2018 :::
      
      select name,col1 from student lateral view explode (array(name,dept,marks,gender)) x as col1 where x.col1="90";

      
 1 . get Distinct genres:::
      
      select distinct  A.* from (select explode( genres) from movies) A  ;
      hive (hdpmovielens)> select distinct  A.g1 from movies lateral view explode( genres) A as g1 ;
      https://www.ericlin.me/2013/09/how-to-use-hive-lateral-view-in-your-query/
        
         select distinct  movies.title,A.g1 
          from movies lateral view explode( genres) A as g1 where A.g1 rlike 'Children\'s'

           select distinct  movies.title,A.g1 from movies lateral view explode( genres) A as g1 where A.g1 like '%me%';
select count(title)  from movies 
lateral view explode( genres) A as g1 where A.g1 like '%me%';
1538

select count(title),g1  
from movies lateral view explode( genres) A as g1 where A.g1 like '%me%'
group by A.g1;
_c0     g1
1200    Comedy
211     Crime
127     Documentary

select title from movies where  array_contains(genres,"Documentary")

      

In [0]:
1. Top ten most viewed movies with their movies Name (Ascending or Descending order)
ratings user id ,movie id rating when ?
movie  mvid and title 

movieid group and count of each movie -> winoding  10 result jon data 

select count(1),movieid from ratings group by movieid;
select count(*),movieid from ratings group by movieid;
select count(movieid),movieid from ratings group by movieid;

select r.movieid, count(r.movieid) cnt
from ratings r
group by r.movieid
order by cnt desc
limit 10;


select r.movieid, count(r.movieid) cnt, m.title
from ratings r join movies m ON r.movieid=m.movieid
group by r.movieid,m.title
order by cnt desc
limit 10;

select A.*,m.title
from 
(select r.movieid, count(r.movieid) cnt
from ratings r
group by r.movieid
order by cnt desc
limit 10) A join movies m ON A.movieid=m.movieid





In [0]:
Date: 19 Sept 2018
   yarn application -list 
   set mapreduce.job.name;

  dense_rank()
 

select B.*,m.title from
(select A.*, dense_rank() over (order by A.cnt desc)  as rnk
from
(select r.movieid, count(r.movieid) cnt
from ratings r
group by r.movieid) A )B 
join movies m ON B.movieid=m.movieid

where B.rnk<=10;
  

In [0]:
Analytic and window function
stu1,dept1,90
stu2,dept2,80
stu3,dept1,60
stu4,dept2,80
stu5,dept1,90
stu6,dept1,60
stu7,dept2,70
stu8,dept1,30
stu9,dept2,40

create table students (id string,dept string,marks int)
row format delimited 
fields terminated by ','
stored as textfile

hdfs dfs -appendToFile - /user/hive/warehouse/hdpmovielens.db/students/stu.txt
hdfs dfs  -cat  /user/hive/warehouse/hdpmovielens.db/students/stu.txt



select *,
rank() over (order by s.marks desc ) as rank,
dense_rank() over (order by s.marks desc ) as drank,
row_number() over (order by s.marks desc ) as rno,

rank() over (partition by dept order by s.marks desc ) as dept_rank,
dense_rank() over (partition by dept order by s.marks desc ) as dept_drank,
row_number() over (partition by dept order by s.marks desc ) as dept_rno
from students s


select * from students
where marks = (select max(marks) 
                from students 
                where marks <> (select max(marks) from students
                )
              );

-----------------------------

Add gender column to students table..


stu1,dept1,90,M
stu2,dept2,80,F
stu3,dept1,60,M
stu4,dept2,80,M
stu5,dept1,90,M
stu6,dept1,60,F
stu7,dept2,70,M
stu8,dept1,30,M
stu9,dept2,40,M

create table students (id string,dept string,marks int)
row format delimited 
fields terminated by ','
stored as textfile;

truncate table students;
alter table students add columns (gender string) ;

dept   male female 
dept1   4   1
dept2   4   1

class M F
      8 2
  
  select sum(if(gender='M',1,0)) male, 
  sum(if(gender='F',1,0)) female
  from students;
  
  
  select dept,sum(if(gender='M',1,0)) male, 
  sum(if(gender='F',1,0)) female
  from students
  group by dept;
  
  case column  
  when A  then do something -> column=A 
  when B 
  else xyz
  end
  
  
  case 
  when column=A then 
  when column=B
  else
  end
  
  select sum(
        
     (case gender 
      when 'M' 
      then 1 
      else 0 
      end)
  
  ) male, 
  sum((case  
       when gender='F' 
       then 1 
       else 0 end)) female
  from students;
  
  
  hdfs dfs -appendToFile - /user/hive/warehouse/hdpmovielens.db/students/stu.txt

  stu10,dept2,NULL,NULL

  Question:
    dept wise avg marks? 
    
    select avg(marks), dept from students group by dept;
  
  hive (hdpmovielens)> select marks,NVL(marks,0) from students;
hive (hdpmovielens)> select marks,NVL(marks,0) from students
where marks is null;

coalesce:
  
HOME WORK::
    Windowning and analytic functions :: on MARKS column
        SUM 
        AVG
        COUNT
        LEAD
        LAG
        FIRST
        LAST
    Solve remainig 3 question
    
    CUSTOM UDF- with examples

In [0]:
20th Sept 2018

hive (hdpmovielens)> select from_unixtime(unix_timestamp()),
current_timestamp();



yyyy-MM-dd HH:mm:ss.SSS
    
    
    if the string is in standard format - convert into other format - 
    1) date_format(date/timestamp/string ts, string fmt)
    
   if dd-MM-yyyy into MM-dd-yyyy
  unix_timestamp(string date, string pattern) into bigint
  from_unixtime(bigint unixtime[, string format])
  
  
  2019-01-31 convert this into 31-01-2019 and 01-31-2019
  select "2019-01-31", 
  unix_timestamp("2019-01-31 00:00:00"),
  from_unixtime(unix_timestamp("2019-01-31 00:00:00"),"MM-dd-yyyy"),
  from_unixtime(unix_timestamp("2019-01-31 00:00:00"),"dd-MM-yyyy"),
  date_format("2019-01-31","MM-dd-yyyy"),
  from_unixtime(unix_timestamp("2019-jan-31","yyyy-MMM-dd"),"dd-MM-yyyy");
  
  
  
  01-31-2019 -> convert this into standard format ::MM-dd-yyyy
      
      select from_unixtime(unix_timestamp("01-31-2019","MM-dd-yyyy"),"dd-MMMM-yyyy");



In [0]:

Cross
dept stu1 stu2 stu3  s5 s6 7 8 9
dept1  90  
dept 2               80 90


select dept, map(name,marks) from student where dept='dept1';

select dept, collect_list(map(name,marks)) from student  group by dept;


dept1   [{"stu1":90,"stu3":60,"stu5":90,"stu6":60},{"stu8":30}]
dept2   [{"stu2":80},{"stu4":80},{"stu7":70},{"stu9":40}]

hive (hdpmovielens)> select dept, collect_list(concat_ws("-",name,cast(marks as string))) from student  group by dept;

create table test as select dept, collect_list(concat_ws("-",name,cast(marks as string))) l1 from student  group by dept;






select dept, collect_list(map(name,marks)) from student  group by dept;




select dept, explode(collect_list(map(name,marks))) from student  group by dept;

drop table stg_students;
create table stg_students as 
select dept, collect_list(map(name,marks)) as stu_list from student  group by dept;


select dept,explode(stu_list) from stg_students;

select dept,col1 from stg_students  lateral view explode (stu_list) V1 as col1 ;



select dept, collect_list(map(name,marks))['stu1'] as "stu1" ,
collect_list(map(name,marks))['stu2'] as "stu2" ,
collect_list(map(name,marks))['stu3'] as "stu3" ,
collect_list(map(name,marks))['stu4'] as "stu4" ,
collect_list(map(name,marks))['stu5'] as "stu5" ,
collect_list(map(name,marks))['stu6'] as "stu6" ,
collect_list(map(name,marks))['stu7'] as "stu7" ,
collect_list(map(name,marks))['stu8'] as "stu8" ,
collect_list(map(name,marks))['stu9'] as "stu9" 
from student group by dept;






In [0]:
3. Top twenty rated movies (which is calculated in the previous step) with no of views in the
following age group
(Age group : 1. Young (<20 years),
2. Young Adult(20-40 years),
3.adult (> 40years) )

drop table q2_ansser;
create table q2_ansser as 
select A.*,m.title from (
select avg(rating) avg_rating,count(r.movieid) cnt, r.movieid , 
    rank() over (order by avg(rating) desc) rnk
from ratings r 
group by r.movieid
having count(r.movieid)>40
    
) A join movies m on A.movieid=m.movieid 
where A.rnk <=20

hive (hdpmovielens)> select a.*,m.name from q2_ansser a join movies m on a.movieid=m.movieid;


age group is present in user data
mvid,userid in ratings
moviid,name is in movies

select * from user





     20     40    60
mvid count  count count
----------------------------------

convert age groups into 1,2,3 
sum (case agegroup when 1 | 18 then 1 else 0) age_20
sum (case agegroup when ,...| 18 then 1 else 0) age_40
sum (case agegroup when 1 | 18 then 1 else 0) age_20

group by movieid from you ratings where ratings.movie id in q2-anser

moviid  count 20
movie    count 40 

convert these values in to cross tab ...


select count(1) from ratings r where r.movieid=318;
2227
g20 , g40, g60 
select u.userid,u.age from users u join ratings r on u.userid=r.userid

where r.movieid=318;

select r.movieid, count(1) cnt, sum(case when u.age=1 or u.age=18 then 1 else 0 end) g20,
       sum(case when u.age=25 or u.age=35 then 1 else 0 end) g40,
       sum(case when u.age=45 or u.age=50 or u.age=56 
           then 1 else 0 end) g60
    from users u join ratings r on u.userid=r.userid
where r.movieid in (select movieid from q2_ansser)
group by r.movieid 


select max(q.avg_rating),q.rnk ,r.movieid, count(1) cnt, sum(case when u.age=1 or u.age=18 then 1 else 0 end) g20,
       sum(case when u.age=25 or u.age=35 then 1 else 0 end) g40,
       sum(case when u.age=45 or u.age=50 or u.age=56 
           then 1 else 0 end) g60, q.title
    from users u join ratings r on u.userid=r.userid
    join q2_ansser q on r.movieid=q.movieid
-- where r.movieid in (select movieid from q2_ansser)
group by r.movieid ,q.title;









In [0]:
AUTOMATION - run this for every 1 hour (cron tab )
as well as on demand 
1) user_ip,ratings_ip,movies_ip

cleansing steps 

q2_answer table 

final output 

prepare shell script to run all in sequence

when each query completes - echo query cleaning data has comleted
-------------
when every step completes remove checkpoing file
statgin create check point file
when the current is already in progress you print a message -> progress current is in progress

----------------------------
source tables 
rating will be partitined key
movieid will be bucketing -> sort 
----------------------------
database names you are going to put in a .properties file
use the file in shell script 
---------------------------------
set the job names properly 
loaddata_date(yyyy_mm_dd_HH_mm_ss)_step1
------------------

RUN the script usiong hive and beeline 
----------------------------------
UDF hi my name is bdt my email id xyz@cisco.com
and my alternate mail id is test@yahoo.co.in 

hi my name is bdt my email id xyz@cisco.com
and my alternate mail id is test@gmail.com




ADD JAR /home/dorababugjntup/Saluatation_UDF.jar;
CREATE TEMPORARY FUNCTION addSalute as 'com.bdt.Salutation';
desc function addSalute;
desc function  extended addSalute;
show functions like '*Sal*;
show functions like '*Sal*';
select name, gender, addSalute(name) , addSalute(gender,name) from student;
select name, gender, addSalute(name) , addsalute(gender,name) from student;
select name, gender, addsalute(name) , addsalute(gender,name) from student;
select name, gender, Addsalute(name) , Addsalute(gender,name) from student;
ADD FILE /home/dorababugjntup/upper.py
;
select transform(name) using 'python upper.py' from student;
select transform(name) using 'python upper.py' as (CAP_NAME)  from student;



package com.bdt;

import org.apache.hadoop.hive.ql.exec.Description;
import org.apache.hadoop.hive.ql.exec.UDF;
import org.apache.hadoop.hive.ql.udf.UDFType;
import org.apache.hadoop.io.Text;

@Description(
	name = "Salute_add", 
	value = "_FUNC_(gender, name) - Adds salutation", 
	extended = "This function will return salutation and name"
)
@UDFType(deterministic = true, stateful = false)
public class Salutation extends UDF {
   public Text evaluate(String str) {
      return str == null ? null : new Text("Mr. "+str);
   }
   public Text evaluate(String gender,String name) {
	   
	   if(gender.equalsIgnoreCase("M")){
			String str=	"Mr. " + name;
			return name == null ? null : new Text(str);
	   }else
		   return name == null ? null : new Text("Mrs. "+ name);
	   
      //return name == null ? null : new Text(str);
   }
   
  /* public static void main(String[] args) {
	   Salutation obj=new Salutation();
	  System.out.println(obj.evaluate("BigDataTech"));
	  System.out.println(obj.evaluate("M","BigDataTech"));
	  System.out.println(obj.evaluate("F", "BigDataTech"));
}*/
   
}


---------------------------------

#!/usr/bin/env python
'''
This is a script to upper all cases
'''
import sys

def main():
    try:
        for line in sys.stdin:
          n = line.strip()
          print n.upper()
    except:
        return None


if __name__ == "__main__":main()




In [0]:
Unix framework to run the Hive queries:::
Project:::: 24 sept 2018
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
      job group -> RDBMS -> Ratings_Group  job_step_run is table name.
pk jobgroupname jobstep description status created_date updated_date
----------------------
insert 
1 ratings_group 1 cleandata P now() now()

1 ratings_group 1 cleandata R samedate now()

1 ratings_group 1 cleandata E samedate now()

1 ratings_group 1 cleandata C samedate now()
---------------------------------------------

write a unix function that takes status and step number
print log -> updating status of step number as  ->status  at time
----------------
step1.hql
step2.hql
step3.hql
-------------------
write another function that runs the step
fnciton (stepname)
hive -f step1
check the exit status
update the status in RDBMs
if success exit 0
if failures update status as E in rdbms
send a mail 
exit 1
------------------------


Begening check entries are present or not, if entries present print previous batch is pending
run the pending steps.
if no previous batch is present -> insert entreis in to control table and execute steps.

At the end of job, check all jobs completed, then delete the entries .
---------------------------
All above is for control table.  job_step_run table 


job_step table ::::: 
Keep another table where each job how many steps are present? 
each step what is the job name and script name and location.
shell script while executing extract the scipt name from these entries and run it using 
hive -f scrpt location.
---------------------------



In [0]:
ACID support in Hive :::: 
24 sept 2018

In [0]:
SQOOP INSTLLATION
https://dorababu-bigdata-trainer.blogspot.com/2018/09/sqoop-installation-in-gcp.html
  
  
SQOOP download : wget http://www-us.apache.org/dist/sqoop/1.4.7/sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
    ls -lh sqoop*
    tar -xvf sqoop-1.4.7.bin__hadoop-2.6.0.tar.gz
    mv sqoop-1.4.7.bin__hadoop-2.6.0 YARN/sqoop
    
    
    vi $HOME/.bashrc
export SQOOP_HOME=$HOME/YARN/sqoop
export PATH=$PATH:$SQOOP_HOME/bin
export SQOOP_CONF_DIR=$SQOOP_HOME/conf
export SQOOP_CLASSPATH=$SQOOP_CONF_DIR

source ~/.bashrc

cp YARN/hive/lib/mysql-connector-java-8.0.11.jar YARN/sqoop/lib/




In [0]:
sqoop list-databases \
--connect jdbc:mysql://localhost:3306/ \
--username root \
--password root 
      
mysql -uroot -proot -e "show databases"

sqoop eval \
--connect jdbc:mysql://localhost:3306/mysql \
--username root \
--password root \
      -e "show tables"
      
       mysql -uroot -proot -e "use mysql;show tables"
        
   sqoop list-tables \
--connect jdbc:mysql://localhost:3306/mysql \
--username root \
--password root 
        
        
        
   Source data set::
      wget https://github.com/dorababugjntup/SET2/raw/master/1_LoadMySQLData.tar.gz

tar -xvf 1_LoadMySQLData.tar.gz
cd 1_LoadMySQLData
sh 1_LoadDataMySQL.sh
user name: root  
  password :: root
   (y/n)small y
  
  
  mysql -uroot -proot
  show databases;
  use complaints_db;
  show tables;
  mysql> select count(1) from complaints;
mysql> select count(1) from usstates;


In [0]:
Date 26 sept 2018
Sqoop import
------------------------
sqoop import \
--connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
--username root \
--password root \
--table usstates \
--enclosed-by '"' \
--delete-target-dir \
--columns id,state \
-m 10 \
--target-dir /11111 \
--where "state like 'A%'" 



--boundary-query "select 1,1000" \

--split-by id \




sqoop import \
--connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
--username root \
--password root \
--enclosed-by '"' \
--delete-target-dir \
--columns id,state \
-m 2\
--split-by id \
--target-dir /11111 \
-e " select u.id,u.state from usstates u where state like 'A%' and \$CONDITIONS "

"select * from tableA ,table B where A.c1=B1.c2 and \$CONDITIONS

SELECT MIN(t1.id), MAX(t1.id) FROM ( select u.id,u.state from usstates u where state like 'A%' and  (1 = 1)  ) AS t1

hive import
incremental import
sqoop job





In [0]:
Date 27 Sept 2018
Create usstates_part table partitioned by start letter - column name is start_name
------------------

Via Sqoop create a table in hdpmovielens - same table name 
and peform full refresh
--------------------------
hdfs dfs -rm -r hdfs://localhost:9000/user/dorababugjntup/usstates 
    cp YARN/hive/lib/hive-exec-2.3.3.jar YARN/sqoop/lib/
    cp YARN/hive/lib/mysql-con* YARN/sqoop/lib/
sqoop import \
--connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
--username root \
--password root \
--hive-import \
--create-hive-table \
--hive-database hdpmovielens \
--table usstates \
--delete-target-dir 


hive (hdpmovielens)> select id,created_date,modified_date,state from usstates;

in mysql::
    insert into usstates (state,standardabbreviation,postalcode,capitalcity) values("aaa","aaa","aaa","aaa");

export lastval=`hive -S -i test.hql -e "select max(id) from hdpmovielens.usstates"`

sqoop import \
--connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
--username root \
--password root \
--hive-import \
--hive-database hdpmovielens \
--table usstates \
--hive-table usstates \
--incremental append \
--check-column id \
--last-value $lastval 

export lastval=`hive -e "select max(id) from hdpmovielens.usstates"`


To store the last vlaue create a sqoop job

sqoop job --list
sqoop job --delete usstates_nonmutable

sqoop job --create usstates_nonmutable \
-- import \
--connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
--username root \
--password root \
--table usstates 
--hive-import \
--hive-database hdpmovielens \
--table usstates \
--hive-table usstates \
--incremental append \
--check-column id \
--last-value 0 \
select * from usstates where id >lastvalue and id <= max(id)

sqoop job --list
sqoop job --show jobid
sqoop job --exec jobid -- --last-value 0 --delete-target-dir --hive-table xyz

--------------------

Mutable Data:: 
    Old records also gets modified ::: 
          Insert / update 
          
          
            sqoop import \
  --connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
  --username root \
  --password root \
  --table usstates \
  --incremental lastmodified \
  --check-column modified_date \
  --target-dir /user/hive/warehouse/hdpmovielens.db/usstates \
  --fields-terminated-by '\001' \
  --append 

           
 
         
            sqoop import \
  --connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
  --username root \
  --password root \
  --table usstates \
  --incremental lastmodified \
  --check-column modified_date \
  --target-dir /user/hive/warehouse/hdpmovielens.db/usstates \
  --fields-terminated-by '\001' \
  --append 
    
    
    update usstates set state="ZZZZ" where state like 'P%';
 insert into usstates (state,standardabbreviation,postalcode,capitalcity) values("bbb","bbb","bbb","bbb");

    

     create table ctrltbl (maxdate string) partitioned by (jobname string);
            
            set hive.exec.dynamic.partition.mode=nonstrict;
            insert overwrite table ctrltbl partition(jobname)
            select max(modified_date),"usstates" from usstates;
            
    
    export lastval=`hive -S -i test.hql -e "select  maxdate from hdpmovielens.ctrltbl"`

    
    hdfs dfs -rm -r /user/hive/warehouse/hdpmovielens.db/usstates_incr
    
        sqoop import \
  --connect "jdbc:mysql://localhost:3306/complaints_db?zeroDateTimeBehavior=CONVERT_TO_NULL" \
  --username root \
  --password root \
  --table usstates \
  --incremental lastmodified \
  --check-column modified_date \
  --target-dir /user/hive/warehouse/hdpmovielens.db/usstates_incr \
  --fields-terminated-by '\001' \
  --last-value "$lastval" \
      --append 


      CREATE TABLE `usstates_incr`(
  `state` string,
  `standardabbreviation` string,
  `postalcode` string,
  `capitalcity` string,
  `created_date` string,
  `modified_date` string,
  `id` bigint)
COMMENT 'Imported by sqoop on 2018/09/27 03:49:48'
ROW FORMAT SERDE
  'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'field.delim'='',
  'line.delim'='\n',
  'serialization.format'='')
STORED AS INPUTFORMAT
  'org.apache.hadoop.mapred.TextInputFormat'
OUTPUTFORMAT
  'org.apache.hadoop.hive.ql.io.HiveIgnoreKeyTextOutputFormat'
LOCATION
  'hdfs://localhost:9000/user/hive/warehouse/hdpmovielens.db/usstates_incr'
TBLPROPERTIES (
  'transient_lastDdlTime'='1538104179')


set hive.exec.dynamic.partition.mode=nonstrict;
            insert overwrite table ctrltbl partition(jobname)
            select max(modified_date),"usstates" from usstates_incr;
            
we got old(ustates) and incr(usstate_incr)

create table usstates_replic as select * from usstates;



insert overwrite table usstates
select B.* from usstates_incr B
union all
select A.* from usstates_replic A where A.id not in
(select incr.id from usstates_incr incr);

insert overwrite table usstates
select D.state,
D.standardabbreviation,
D.postalcode,
D.capitalcity,
D.created_date,
D.modified_date,
D.id

from (
select 
    C.* , 
    row_number() over 
    (partition by id order by modified_date desc) as rno
  from (
  select B.* from usstates_incr B
  union all
  select A.* from usstates_replic A ) C
) D where D.rno=1;



insert overwrite table usstates_replic as select * from usstates;
dfs -rm -r usstates_replic;
dfs -cp usstates  usstates_replic;
hive>msck repair table usstates_replic;
-----------------------------

1) original - replic table, INCR table 
2) control table to store max date from incr table

3) full refresh and as soon as data refreshed in original, 
overwrite replic table
----- INCR steps
4) sqoop into incr table
5) update control table
6) Removal of duplicates - take repli and incr -> 
insert overwrite original
(if the table is partitioned , remove diuplciates only from the partitions preset in 
replic)
7) copy the data from original to replic
-------------------




In [0]:
APACHE SPARK - 3rd Oct 2018
---------------------------------

5th Oct - RDD properties


wget http://www-eu.apache.org/dist/spark/spark-2.3.2/spark-2.3.2-bin-hadoop2.7.tgz
 tar -xvf spark-2.3.2-bin-hadoop2.7.tgz
mv spark-2.3.2-bin-hadoop2.7 YARN/spark

vi .bashrc
export SPARK_HOME=$HOME/YARN/spark
export PATH=$PATH:$SPARK_HOME/bin

source .bahsrc 
----------------
ls -l YARN/spark
which spark-shell


In [0]:
SCALA PROGRAMMING
Data type hierarchy
Variable Types val var lazy val


Type Inference - identify the data type of a variable from assignment value.
Type Safety - we can assign a varaibel with the same type 

In [0]:
FUnctional programming:::
      1) functions are firstclass citizer, they can exists any where in sourceode
      with name , without a name , accept args, return some value 
      function is code block between { }
      
      2) functions are objects -> assign a function to a variable 
      3) functions can be passed to other functions/methods and/or returns functions
      the methods/funcitons that accept or return a functon are called higher order functions/methods
    
    Call by name and call by value ? what is the difference::
        
        
        {
            val x=10
            val y=100
            println("sum of x and y is "+(x+y))
            x+y
        }
        
        => transformation / rocket symbol
 val sum= (x:Int,y:Int ) => {
            println("inside sum function ---sum of x and y is "+(x+y))
            x+y
        }:Int
        
  sum: (Int, Int) => Int = <function2>

 def sum_method(x:Int,y:Int):Int={
       println("inside sum-method---sum of x and y is "+(x+y))
            x+y
 }
  sum_method: (x: Int, y: Int)Int
-------------------------------------------
Write sub function::
    val sub=(x:Int,y:Int)=>{
         println("inside sub function ---sub of x and y is "+(x-y))
            x-y
    }
        
    sub: (Int, Int) => Int = <function2>
-----------------------------
Higher order method::::
        
        def operation(a:Int,b:Int,fn:(Int,Int)=>Int)={
            println("inside opearatioin method")
            fn(a,b)           
        }
        
        val operation_fn=(a:Int,b:Int,fn:(Int,Int)=>Int)=>{
            println("inside opearatioin method")
            fn(a,b)           
        }
      
      operation(21,20,(x,y)=>x%y)

        
         operation(10+20,sum_method(100,200),sum)
          
          operation(30,300,sum)-> 10+20 , call sum_methdo->go inside operation->
          println inside operation -> 
          code for sum replaced-> inside fun -> sum(30,300)->result
          
          ==================================
          call by value and call by name -> collection types in scala
          tuple
          array
          range
          list
          map
          string
          diff b/n array and vector 
          
              https://github.com/dorababugjntup/SET2/blob/master/SqFluOozHBase.zip

In [0]:
Scala Collections - 9 Oct 2018
import scala.collections.mutable._
 scala.collections.immutable is already imported.
  
  tuple ->  to pack multiple data types , it is not a collection -> no index/loops
  scala> val t=(1,"stu1",200.50,Array("sub1","sub2"))

  scala> val a2=Array(10,true)
  
 val a= Array(10, 20, 30, 40)
 a(0)=10000
  a.foreach((x:Int)=>{println(x)})
a.foreach(x=>println(x))
a.foreach(println)
 val stu=Array("stu1","stu2","stu3")
scala> val fn=(x:String)=> {
     | val y=x+"***"
     | println(y)
     | y
     | }
fn: String => String = <function1>
  scala> stu.map(fn)
   stu.map(x=>{val y=x+"***";println(y);y})

 stu.map(x=>x+"***")
stu.map(x=>x+"***").foreach(println)
-------------------------------------------------------------------
val t=(1,"stu1",200.50,Array("sub1","sub2"))
t._1
t._4
t._4.(0)
t._4(0)
t
t.toString
t.toString.charAt(0)
val t1=(10,"stu1")
t._2
val t=(1,"stu1",200.50,Array("sub1","sub2"))
val t1=(10,"stu10")
t1.swap
t
val s1=(t._2,t._3)
val a=Array(10,20,30)
val a1=Array("str1","st2")
val a2=Array(10,true)
val a2=Array(10,"stu1")
val a2:Array[Any]=Array(10,true)
val a2=Array("stu1",List(10,20))
val a2=Array(List(10,20),List(10,20))
val a2=Array(List(10,true),List(10,20))
val a2=Array(Array(10,true),List(10,20))
val a2=Array(Array(10,true),List(10,20),Map("hi"->"stu1")
)
t.productIterator
for (a <- t.productIterator) println(a)
for (a <- t.productIterator) println(a.getClass)
t
val a=Array(10,true,'c')
val a=Array(10,true,"c")
t.
t
a
val a=Array(10,20,30,40)
a(0)
a
a(3)
a(4)
a.length
a
a(0)=10000
a
a=Array(10,20)
for( elem <- a ) println(elem)
a.foreach((x:Int)=>{println(a)})
a.foreach((x:Int)=>{println(x)})
a.foreach(x=>println(x))
a.foreach(println)
val b=a.foreach(x=>println(x+10))
b
val stu=Array("stu1","stu2","stu3")
stu.foreach(println)
a
val b=a.map(x=>x=10)
val b=a.map(x=>x+10)
b
val b=a.map(x=>x+"hi")
val b=a.foreach(x=>x+"hi")
stu
stu.map(x=>10)
stu.map(x=>{println(x)})
stu.map(x=>{val y=x+"***";println(y);y})
vla fn=(x:String)=> {
val y=x+"***"
println(y)
y
}
val fn=(x:String)=> {
val y=x+"***"
println(y)
y
}
stu.map(fn)
 stu.map(x=>{val y=x+"***";println(y);y})
stu.map(x=>x+"***")
stu.map(x=>x+"***").foreach(println)


In [0]:
RANGE::
    val r= 1 to 10 
 MAP val m=Map("maths" -> 90 , "science" -> 80)
TUPLE
ARRAY
LIST::: 
      (1 to 10 ).toList
      List(1,2,3)
      1::2::3::Nil
                  
   Create a list with every element having id,name, fee, gender
  with 5 students
  
  val stu=List((3,"stu3",500.40,"F"),(1,"stu1",100.40,"M"),(5,"stu5",700.40,"M"),
               (2,"stu2",200.40,"F"),
               (4,"stu4",100.40,"M"))
      
  Operators:: 
      => transformation or rocket to define function
      -> map key value separator
      <- generator 
      . object.method
      :: append 
      : -> variable:data type
          
      https://www.scala-lang.org/api/2.12.3/scala/collection/immutable/List.html
      
      
      
      623  stu
624  l.foreach(println)
625  stu
626  stu.foreach(x=> println(x._2))
627  (3,stu3,500.4,F)._2
628  (3,"stu3",500.4,"F")._2
629  stu.map(x=>(x._1,x._3))
630  stu
631  l
632  l.map(_ + 10)
633  l.map(x =>x+ 10)
634  l.map(x =>x+ "10)"
635  l.map(x =>x+ "10")
636  l.map(x =>(x,x+10))
637  l.map(x=> if(x%2==0) x)
638  l.map(x=> if(x%2==0) x).size
639  l.filter(x=> x%2==0 )
640  l.filter(x=> x%2!=0 )
641  l.filterNot(x=> x%2==0 )

           
           
   647  import scala.collection.mutable.ListBuffer
648  val l1=ListBuffer(1,2,3,4,5)
649  l1.drop(2)
650  l1 += 1000
651  l1


In [0]:
11-Oct-2018::
    foreach - it takes a function with one arg and return unit
     x => UNIT 
    map -> return a collection by applying a funciton on source collection
  no of elements and collectionm type will be same, 
  element data type can change
  filter -> x => boolean 
  
  val l=(1 to 10).toList
  val stu=List((3,"stu3",500.40,"F"),(1,"stu1",100.40,"M"),(5,"stu5",700.40,"M"),
               (2,"stu2",200.40,"F"),
               (4,"stu4",100.40,"M"))
  
   val ml=stu.filter(x=>x._4.equalsIgnoreCase("m"))
     val fl=stu.filterNot(x=>x._4.equalsIgnoreCase("m"))
    val (ml,fl)=stu.partition(x=>x._4.equalsIgnoreCase("m"))
     l.partition(x=>x%2==0)._1 

Question::       "print name,gender of male students with fee < 500"

      
      scala> stu.filter(x=>x._4.equalsIgnoreCase("m")).filter(x=>x._3 < 500).map(t=>(t._2,t._4))
res45: List[(String, String)] = List((stu1,M), (stu4,M))

scala> stu.filter(x=>x._4.equalsIgnoreCase("m") && x._3 < 500).map(t=>(t._2,t._4))
res46: List[(String, String)] = List((stu1,M), (stu4,M))
  
  stu.
  filter(x=>x._4.equalsIgnoreCase("m")).
  filter(x=>x._3 < 500).
  map(t=>(t._2,t._4)).
  foreach(println)

  ------------
  sort student list based on fee paid::
      scala> stu.sortBy(t=>t._3).foreach(println)
(1,stu1,100.4,M)
(4,stu4,100.4,M)
(2,stu2,200.4,F)
(3,stu3,500.4,F)
(5,stu5,700.4,M)

scala> val l1=List(100,30,4,2,80)
l1: List[Int] = List(100, 30, 4, 2, 80)

scala> l1.sortBy(x=>x)
res62: List[Int] = List(2, 4, 30, 80, 100)

  Sort in reverse based on fee :: 
      stu.sortBy(t=>t._3).reverse.foreach(println)

 stu.sortBy(t=>t._3)(scala.math.Ordering[Double].reverse).foreach(println)
(5,stu5,700.4,M) => x._4
(3,stu3,500.4,F)
(2,stu2,200.4,F)
(1,stu1,100.4,M)
(4,stu4,100.4,M)

l1.sortWith((x,y)=>{println(x+":::"+y);x>y})
scala> stu.sortWith((t1,t2)=> t1._3>t2._3).foreach(println)
scala> stu.sortWith((t1,t2)=> t1._3<t2._3).foreach(println)
scala> stu.reduceLeft((t1,t2)=>(0,"",t1._3+t2._3,"") )._3

scala> l1.scanRight(0)((x,y)=>{println(x+":::"+y);x+y})

scala> l1.foldLeft(0)((x,y)=>{println(x+":::"+y);x+y})
scala> l1.reduceLeft((x,y)=>{println(x+":::"+y);x+y})


In [0]:
scala - fiels rdbms
spark files rdbms
spark rdds data frames
spark-sumit
spark sql
spark+hive
flume+kafkak+streaming
hbase




In [0]:
scala> l1.zipWithIndex
scala> l1.zipWithIndex.unzip
scala> val l2=(1 to 4).toList
scala> l1.zip(l2)

scala> l2.zipAll(l1,1000,"$")
scala> l2.zipAll(l1,1000,"$")
l1.mkString("start",end="end",sep="$")

union
scala> "hi welcome".indexOf("w")


val s1=List("hi welcome to class","spark and scala")
s1.map(line=>line.split(" "))
 s1.map(line=>line.split(" ")).flatten
s1.flatMap(line=>line.split(" ")).foreach(println)

word count:::
      val s1=List("hi welcome to class","spark and scala","spark and scala")
      s1.flatMap(line=>line.split(" ")).groupBy(x=>x)
      Map(key,vlaue) -> word , list(words) -> map x(hi,list(hi,hi,hi))=> x._1,x._2.size
      s1.flatMap(line=>line.split(" ")).groupBy(x=>x).map(t=>(t._1,t._2.size))
      
      
      val stu=List((3,"stu3",500.40,"F"),(1,"stu1",100.40,"M"),(5,"stu5",700.40,"M"),
               (2,"stu2",200.40,"F"),
               (4,"stu4",100.40,"M"))
  
Display M,F student count
stu.groupBy(t=>t._4).map(t=>(t._1,t._2.size)).foreach(println)
check how many M studetns present
stu.count(t=>t._4=="M")
get distinct values from collection
stu.map(t=>t._4).distinct
stu.map(t=>t._4).toSet

Q) instead of group by use distinct and count 
scala> for(a<-stu.map(t=>t._4).distinct ) println(a+":::"+stu.count(t=>t._4==a))

scala> for(a<-stu.map(t=>t._4).distinct ) yield(a,stu.count(t=>t._4==a))
for(a<-stu.map(t=>t._4).distinct ) yield(a->stu.count(t=>t._4==a))

import scala.collection.mutable.StringBuilder
val sb=new StringBuilder("test")
 l.addString(sb)

  
  ------------------------
  Read data from File
  
1171  import scala.io.Source
scala> for(line <- Source.fromFile("empdata").getLines) println(line)

1154  Source.fromFile("test").mkString.split(";")(1)
1155  Source.fromFile("test").mkString.split(";")(3)
1160  Source.fromFile("test").toList   // charcters into list
1162  Source.fromFile("test").mkString
1163  Source.fromFile("test").getLines
1164  Source.fromFile("test").getLines.foreach(println)

val lines = Source.fromFile("/Users/Al/.bash_profile").getLines.toList  // all lines into List
val lines = Source.fromFile("/Users/Al/.bash_profile").getLines.toArray



https://alvinalexander.com/scala/scala-jdbc-connection-mysql-sql-select-example

In [0]:
SPARK RDDS::
    
    We need collections(distributed) to hold data.
    Spark -can be considered as  separate language -> have its own data type
    Collectins to hold data in spark are::
        RDD(unstructed data, created using SparkContext)
        DF/DS( to hold structured data , created using SparkSession)
        
    
    spark-shell --help
    --master local[*] take all cores of cpu and create driver, driver itself acts as exectutor
    --deploy-mode client
    --conf PROP=VALUE --conf PROP=VALUE  --conf PROP=VALUE 
    --properties-file FILE 
    --driver-memory MEM
    --driver-java-options
     --executor-memory MEM 
      --driver-cores NUM
      YARN-only:
  --queue QUEUE_NAME 
  --num-executors NUM
  
  
  When Spark-shell is running, it exposes services by WEB UI with externalIpAddress:4040
  enable firewall rules to access this UI.
  (vpc network->filrewall rules->create firewall rule->0.0.0.0/0 and protocols tcp:4040,4041,4042)
  
  Transformations:
  RDD.map , filter, flatMap, -> PairRDD if every element is k-v pair RDD(key,value) 
  pair RDD have extra methods -> Joins are available only in pairRDD, key based operations
  Actions
  RDD.first,count,saveAsTextfile, 
  cache vs persist
  
  
  
  in Spark UI::
      JOB -> Stage-> tasks
      SHuffle read/shuffle Write

In [0]:
16Oct2018
Create RDD -> load from external data sets (textFile,Sequencefiles, hadoop files , object)
or parallelize in memory sequence. or apply a transformatiin on RDD

Create DF/ or DS -> load from stuctrured sources, RDBMS, orc,parquet
or convert RDD by providing schema 
or apply sql in any language , output is DF
or apply DF operations(DSL) creates DF

In [0]:
1513  inputRDD.cache
1515  inputRDD.persist( org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK_SER)
1516  inputRDD.getStorageLevel
1517  inputRDD.unpersist -> fials
1518  inputRDD.unpersist(true)
1519  inputRDD.getStorageLevel
1520  inputRDD.persist( org.apache.spark.storage.StorageLevel.MEMORY_AND_DISK_SER)
1521  inputRDD.getStorageLevel
1522  inputRDD.unpersist(true)
1523  inputRDD.getStorageLevel
-----------------
1526  inputRDD.checkpoint
1527  sc.getCheckpointDir
1528  sc.setCheckpointDir("OutFiles/chkpointdir")
1529  sc.getCheckpointDir
1530  inputRDD.checkpoint
1531  inputRDD.count
r1.getCheckpointFile

In [0]:


val inputRDD=sc.textFile("wc_input.txt")
val r1=inputRDD.flatMap(line=>line.split(" ")).groupBy(x=>x).map(t=> (t._1,t._2.size))
r1.saveAsTextFile("OutFiles/wc_input")

val inputRDD1=sc.textFile("wc_input.txt")
scala> inputRDD1.flatMap(line=>line.split(" ")).groupBy(k=>k).map(t=>(t._1,t._2.size)).foreach(println)
scala> inputRDD1.flatMap(line=>line.split(" ")).map(word=>(word,1)).reduceByKey((x,y)=>x+y).foreach(println)

inputRDD1.
flatMap(line=>line.split(" ")).   // convert line into words and flatte
map(word=>(word,1)).              // convert RDD into PairRDD
reduceByKey((x,y)=>x+y).          // take two values for a key and apply sum and repeat until there no aggregtation possibe;
foreach(println)                  



In [0]:
emp
1,emp1,2000,dept1
2,emp2,6000,dept2
3,emp3,500,dept1
4,emp4,7000,dept2
5,emp5,10000,dept3
6,emp6,4000,dept3

dept
dept1,hive
dept2,oozie
dept4,spark

1) display all emp record, dept name 
val e=sc.textFile("emp")
val d=sc.textFile("dept")
val e_pair=e.map(line=>(line.split(",")(3),line))
val d_pair=d.map(line=>(line.split(",")(0),line.split(",")(1)))
e_pair.join(d_pair).foreach(t=>println(t._2))
d_pair.join(e_pair).foreach(println)

e_pair.leftOuterJoin(d_pair)
e_pair.leftOuterJoin(d_pair).foreach(line=>println(line._2._1+","+line._2._2.getOrElse("NotFound")))

join - dept, (all records for emp, all records for dept)
full outer join  dept(option, option)
left outer dept, emp, option (dept)
right outer dept, ption()emp, (dept)

https://github.com/dorababugjntup/Assignments/tree/master/3_MovieLens
  
  1)Download scala plugin in eclipse market place
  2) Download spark in windows
  
  download SBT in windows and install
  sbt plugin eclipse (sbt project folder structure)
  
  download git in windows
  
  practice maven java (hellow world ) in eclipse

In [0]:
BroadCast Variables::
    1771  val pws = Map("Apache Spark" -> "http://spark.apache.org/", "Scala" -> "http://www.scala-lang.org/")
1772  val searchWords=sc.parallelize(Seq("Apache Spark","Scala" ))
1773  searchWords.map(x=>pws(x)).collect
1774  searchWords.map(pws).collect
1779  val pwsB=sc.broadcast(pws)
1780  searchWords.map(pwsB.value).collect




val e=sc.textFile("emp")
val d_pair=scala.io.Source.fromFile("dept").getLines.map(line => (line.split(",")(0),line.split(",")(1))).toMap
val e_pair=e.map(line=>(line.split(",")(3),line))
//val d_pair=d.map(line=>(line.split(",")(0),line.split(",")(1)))
e_pair.map(t=>(t._2,d_pair.getOrElse(t._1,"Not Found"))).foreach(println)

val e_pair=e.map(line=>(line,d_pair.getOrElse(line.split(",")(3),"Not Found")))


val d_pair_B=sc.broadcast(d_pair)
e_pair.map(t=>(t._2,d_pair_B.value.getOrElse(t._1,"Not Found"))).foreach(println)
-----------------------------------------
val e=sc.textFile("emp")
val d_pair=scala.io.Source.fromFile("dept").getLines.map(line => (line.split(",")(0),line.split(",")(1))).toMap
val e_pair=e.map(line=>(line,d_pair.getOrElse(line.split(",")(3),"Not Found")))
e_pair.foreach(println)
  
  
Map(dept1 -> hive, dept2 -> oozie, dept4 -> spark)
  (5,emp5,10000,dept3,Not Found)
(1,emp1,2000,dept1,hive)
(6,emp6,4000,dept3,Not Found)
(2,emp2,6000,dept2,oozie)
(3,emp3,500,dept1,hive)
(4,emp4,7000,dept2,oozie)  
  
  
  emp left join dept -
  val d_pair_B=sc.broadcast(d_pair)
val e_pair=e.map(line=>(line,d_pair_B.value.getOrElse(line.split(",")(3),"Not Found")))
e_pair.foreach(println)
-----------------------------------------

NOTE:::: We cannot Borad Cast  RDD, we can braodcast Data Frame and other non distributed collections
        Advantages::: reduces n/w traffic and can perform mapside joins
        
        
        without BV -> When ever an executor is started , the value will be sent through n/w to that node -increase n/w traffic
with BV -> the variable is already sent once to the node , whenever new executors starts, no need to transfer the data,
as it is already present in every node.
Map side joins ( send second data set to every node and each task can use this data set for join)


        
        ---------------------
        Accumulator:::
              They can be used to implement counters (as in MapReduce) or sums. 
              it acts like a global variable
              they are shared variables and value can be changed
              val x=sc.longAccumulator("noOfLines")
              val e=sc.textFile("emp")

              e.foreach(line=>{x.add(10);println(line+"***"+x.count)})
              
              We can see the accumulator in Tasks of Stages page.
              This varibale is alive till the session is active.
              
              To find sum of array
              sc.parallelize(Array(1, 2, 3, 4)).foreach(x => accum.add(x))
              accum.value gives 10.
              
Spark---------------------------

In [0]:

SPARK DATA FRAME / DATA SET -> TABLE - we can apply RDD methods, DF/DS methods(DSL), SQL 
its an abstraction (collection in spark)to hold structured data.

1.x -> DF -> RDD + SCHEMA 
How to create data frames??
1) convert exising RDD , and provide schema rdd.toDF(schema)
2) load data from variety of sources
csv,json,RDBS,orc,parque, hive table, textfile
-> 1.x to read csv we need to use external libraries databricks apis
2.x all common formats to read, no need to external libraries
"SparkSession.read" API
3) any language , if you run sql ( DF=sparkSession.sql(query)), the output is Data frame
4) apply any transformation (rdd,df dsl) that creates another DF
5) convert DS into DF

What is the difference between DS and DF
http://spark.apache.org/docs/latest/sql-programming-guide.html

val e=sc.textFile("emp")
val d=sc.textFile("dept")
val e_pair=e.map(line=>(line.split(",")(3),line))
val d_pair=d.map(line=>(line.split(",")(0),line.split(",")(1)))


In [0]:
Date 18th Oct 2018
convert RDD into DF:::
      val fn=(line:String)=>{
          val arr=line.split(",")
          (arr(0).toInt,arr(1),arr(2).toInt,arr(3))
      }
      fn("1,emp1,2000,dept1")
val empRDD=sc.textFile("emp").map(fn)
val empDF=empRDD.toDF("id","name","sal","deptid")
  
  
  empDF -> org.apache.spark.sql.DataFrame - We dont have DataFrame api in scala,
  DF is Dataset[Row], 
  DS is https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.sql.Dataset
    Dataset is strongly typed - type checking at compile time
    DataFrame is untyped view that is present in Dataset, type checking is at run time
    
  How to Create Dataset??? 
  RDD(jvmObjects)
  
  
  case class Emp(id:Int,name:String,sal:Int,deptid:String)
  val fn1=(line:String)=>{
          val arr=line.split(",")
          Emp(arr(0).toInt,arr(1),arr(2).toInt,arr(3)) // setting the values for Emp Object
      }
      fn1("1,emp1,2000,dept1") -> Emp jvm object
    
 val empRDD1=sc.textFile("emp").map(fn1)  -> RDD of JVM objects (Strongly typed )
 val empDS=empRDD1.toDS()

for toDF-> we need to supply column names - def toDF(colNames: String*): org.apache.spark.sql.DataFrame 
for toDS-> no args - def toDS(): org.apache.spark.sql.Dataset
  first create RDD(jvmObjects) then convert using toDS

  --------------------------------------------------
  Create your own schama and load data using this schema
  import org.apache.spark.sql._
import org.apache.spark.sql.types._

val struct = StructType(
Seq(StructField("id",IntegerType,false),
StructField("name",StringType,true), 
StructField("sal",IntegerType,false), 
StructField("deptid",StringType,true))
) 
  
  
  
  import org.apache.spark.sql.Row
  val fn2=(line:String)=>{
          val arr=line.split(",")
          Row(arr(0).toInt,arr(1),arr(2).toInt,arr(3)) // setting the values for Emp Object
      }
      fn2("1,emp1,2000,dept1")
    res9: org.apache.spark.sql.Row = [1,emp1,2000,dept1]
val empRDD2=sc.textFile("emp").map(fn2)   // Create RDD of Row objects and privde your own schema
 val empDF2=spark.createDataFrame(empRDD2,struct)
 ---------------------------------------------------

Summary::: create DF/DS from RDD(unstructured data)
1) RDD.toDF(string column names) ->DF  -> not strictly typed
RDD(jvmobject).toDS() -> first create RDD with jVM objects -> Strict Type checking
(case class , every row of RDD is the object of case class)

2)createStructType,
RDD(row objects) and use spark.createDataFrame(RDD(row),structType)
RDD(jvmobject) and use spark.createDataset(RDD(jvmobjects))
spark.createDataset(empRDD1) same as empRDD1.toDS()
-------------------------------------------------


In [0]:
DataSet Operations::: (DSL -> domain specifig language)
      1) Actions
      2) Typed Transformations - return Dataset
      3) UnTyped Transformatios - return DataFrame
      4)Basic Dataset functions 

In [0]:
Demo of DF operations:: Create data frame::
       val fn=(line:String)=>{
          val arr=line.split(",")
          (arr(0).toInt,arr(1),arr(2).toInt,arr(3))
      }
      fn("1,emp1,2000,dept1")
val empRDD=sc.textFile("emp").map(fn)
val empDF=empRDD.toDF("id","name","sal","deptid")
#### use inmemory metastore
#### If we save the DF as table -> uses derby metastore
#### if we integrate with Hive metastore, we can access same tables in hive and spark.

ACTIONS::
    empDF.count()  , describe().show() , show(10)
    collect(), take(10) 
DataSet functions
     cache
     checkpoint
    empDF.columns
    createGlobalTempView
    createorReplaceGlobalTempView
    createOrReplaceTempView
    createTempView
    
    Procedure to apply SQL on DataFrames::
        empDF.createOrReplaceTempView("emp_table") //inmemory metastore
        now empDF can be accessed in sql using a table name emp_table
        spark.sql("select * from emp_table") -> returns a DF 
        how to store values into a concrete table 
        spark.sql("create table xyz as select * from emp_table") -> returns a DF but unit..
        now xyz table is stored in (since not integrated with hive or hdfs) -> stored in local FS with Derby metastire
        and can be retrieved later.
        2018-10-18 05:17:16 WARN  HiveMetaStore:1383 - 
        Location: file:/home/dorababugjntup/spark-warehouse/xyz specified for non-external table:xyz
rest all DS functions
and typed/untyped functions.
--------
various others sources to read data and create DF/DS

trait,object,class,case class
privamry vs auxillary consturos (overloaded consturctors)
companioin object
partial function
Inheritance and multiple inheritance in Scala
exception handling

write standalone scala spark applicaiton and submit to production
What is encoders and spark.implicits
    

In [0]:
Create DataFrame -> RDD to DF or DS
create df/ds from various sources. -> sparkSession.read()
1) CSV 

id,name,sal,deptid
1,emp1,2000,dept1
2,emp2,6000,dept2
3,emp3,500,dept1
4,emp4,7000,dept2
5,emp5,10000,dept3
6,emp6,4000,dept3
7,emp7,1000
8,8000,dept4

spark.read
   def read: org.apache.spark.sql.DataFrameReader
spark.read.format("csv").load("emp_header")
or spark.read.csv("emp_header")


val df = sqlContext.read
    .format("com.databricks.spark.csv")
    .option("header", "true") 
    .option("mode", "DROPMALFORMED")
    .load("csv/file/path"); 
    
    
    
    val empDF=spark.read.
    format("csv").
    option("sep",",").
    option("header","true").
    option("inferSchema","true").
    option("mode","PERMISSIVE").
    load("emp_header")
    empDF.show()
    
    
    import org.apache.spark.sql.types._
   val schema1=
    StructType(Seq(
              StructField("id1",IntegerType,true),
               StructField("name1",StringType,true), 
              StructField("sal1",IntegerType,true), 
               StructField("deptid1",StringType,true))
              )
    
val empDF=spark.read.
    format("csv").
    option("sep",",").
    option("header","true").
    option("inferSchema","true").
    option("mode","PERMISSIVE").
    schema(schema1).
    load("emp_header")
    
    ---------------------------------------
    
    val empDF=spark.read.
    format("json").
    option("mode","PERMISSIVE").
    load("/home/dorababugjntup/YARN/spark/examples/src/main/resources/people.json")
    
     empDF.na.drop(1).show
atleast one not null value should be there.
scala> empDF.na.fill("NAN",Seq("gender")).show
scala> empDF.na.replace(List("gender"),Map("Male"->"N/A"))

Typed oPs::
empDF.filter(empDF("age")> 19).show
scala> empDF.filter("age = 19").show

Untyped Ops
2366  empDF.agg(Map("age"->"max")).show
2367  empDF.agg(Map("age"->"min")).show
2368  empDF.agg(Map("age"->"avg")).show

empDF.drop(empDF("name")).show
scala> empDF.drop("name").show

scala> def test(c1:String, cols:String*)= println(c1+"***"+cols.length)
test: (c1: String, cols: String*)Unit

scala> test("a","b","c")
a***2



In [0]:
Eclipse http://www.eclipse.org/downloads/download.php?file=/technology/epp/downloads/release/2018-09/R/eclipse-dsl-2018-09-win32-x86_64.zip&mirror_id=272
  Help->Eclipse MarketPlace
  find -> scala -> scala ide 1.7.x
  -----------------------------------
  next download sbt and set path in windows
  mkdir project2
  
  cd project2
  
  create build.sbt inside put scalaVersion := "2.11.11"
    
    mkdir project
  touch plugin.sbt
  
    sbt eclipse
    now .classpath got created with 2.11
    
    Change Scala Library Container to 2.11 version
    
   

In [0]:
Date 22 Oct 2018
Spark Integration with HDFS and HIVE
1) To read data from hdfs and submit jobs to yarn, we need NN and RM address
present in site xml files
export HADOOP_CONF_DIR=$HADOOP_HOME/etc/hadoop
2) To Read from Hive Tables, we just need hive metastore details
copy hive-site.xml  into YARN/SPARK/conf
cp YARN/hive/conf/hive-site.xml YARN/spark/conf/
place mysql connector jar driver in spark/jars
cp YARN/hive/lib/mysql-connector-java-8.0.11.jar YARN/spark/jars/


when using spark and hive with same hive metastore, add below property in hive-site.xml
<property>
    <name>hive.metastore.schema.verification</name>
    <value>false</value>
</property>


Start hdfs daemons
hadoop-daemon.sh start namenode &
hadoop-daemon.sh start datanode &
hadoop-daemon.sh start secondarynamenode &
yarn-daemon.sh start resourcemanager &
yarn-daemon.sh start nodemanager &

----------------
scala>spark.sql(" sql query ")
spark-sql is like hive command
spark-sql --help , at the end we get hive cli commands

export var=`spark-sql -S -i test.hql -e "show tables"`
-------------------------------

Createting tales in spark-shell access in spark-sql and hive
val emp=spark.read.format("csv").option("header","true").load("file:///home/dorababugjntup/emp_header")
emp.show()
emp.drop(emp("deptid")).show
emp.drop("id","deptid").show

emp.createOrReplaceTempView("emp_table")

Data Frame store into HDFS::::
--------------------------
emp.write. 
bucketBy   format       jdbc   mode     options   parquet       save          sortBy
csv        insertInto   json   option   orc       partitionBy   saveAsTable   text

csv,orc,json,jdbc,saveAsTable hive table name
emp.write.format("csv").save("/emp_csv_spark")

emp.write
   def write: org.apache.spark.sql.DataFrameWriter[org.apache.spark.sql.Row]
emp.write.format("csv").option("header","true").mode("overwrite").save("/emp_csv_spark")
emp.write.format("csv").option("header","true").option("sep","#").mode("append").save("/emp_csv_spark")
emp.write.format("csv").option("header","true").option("sep","#").mode("overwrite").partitionBy("deptid").save("/emp_csv_spark")
        
scala> emp.write.saveAsTable("default.emp")
for Hive it istaking ORC as default

scala> emp.write.save("/emp_spark")
for spark defult file format is PARQUET
hdfs dfs -cat /emp_spark/*   give parquet
val emp1=spark.read.load("/emp_spark")


In [0]:
download spark in windows
download scala in windows, extract and set SCALA_HOME and PATH
download sbt  in windows extract and set SBT_HOME and path
jdk download::::
http://download.oracle.com/otn-pub/java/jdk/8u191-b12/2787e4a523244c269598db4e85c51e0c/jdk-8u191-windows-x64.exe
download GIT
Eclipse http://www.eclipse.org/downloads/download.php?file=/technology/epp/downloads/release/2018-09/R/eclipse-dsl-2018-09-win32-x86_64.zip&mirror_id=272
  Extract , open the eclipse 
  Help->Eclipse MarketPlace
  
  find -> scala -> scala ide 1.7.x  -> install , this creates scala SDK in eclipse
  -----------------------------------
  
  
  eclipse-java-oxygen-3a-win32-x86_64 and scala SDK working in my system
  http://www.eclipse.org/downloads/download.php?file=/technology/epp/downloads/release/oxygen/3a/eclipse-dsl-oxygen-3a-win32-x86_64.zip

In [0]:

24 OCT 2018

https://dorababu-bigdata-trainer.blogspot.com/2018/10/git-essentials.html
https://github.com/datamitra/scala-code/blob/master/ScalaDemo/src/HelloWorld.scala

Create Eclipse projects -> prject structure ??
SBT  file reading scala code


compoile package ,run the jar file supply file name arguments -> print file content

--------
MAVEN  scala project 
-----------
GIT in windows make the code availbe in GCP 
---------------

In [0]:
mkdir SparkDemo
build.sbt

scalaVersion := "2.11.11"
  
  echo scalaVersion := "2.11.11">build.sbt
  
  
  Use sbt version 0.13
  C:\Users\username\.sbt\0.13\plugins\plugins.sbt
    addSbtPlugin("com.typesafe.sbteclipse" % "sbteclipse-plugin" % "3.0.0")

  https://github.com/sbt/sbteclipse
    

In [0]:
build.sbt

name := "Simple Project"

version := "1.0"

scalaVersion := "2.11.11"

libraryDependencies += "org.apache.spark" %% "spark-core" % "2.3.2"
libraryDependencies += "org.apache.spark" %% "spark-sql" % "2.3.2"
libraryDependencies += "org.apache.spark" %% "spark-hive" % "2.3.2" % "provided"

In [0]:
https://alvinalexander.com/source-code/scala/common-sbt-commands-scala-sbt
  

In [0]:
package com.bdt

import org.apache.spark.sql.SparkSession
import org.apache.spark.SparkConf
import org.apache.spark.SparkContext


object SparkDemo {
  def main(args: Array[String]): Unit = {
    
  
  val conf=new SparkConf();
  val spark=SparkSession
  .builder()
  .appName("Sample Spark")
  .config(conf)
  .config("spark.app.name","Spark Demo ")
  .master("local[*]")
    .getOrCreate()
  
  val sc=spark.sparkContext
  
  val df1=spark.range(100)
  df1.show(5)
  }
}

In [0]:
Maven Spark Project
https://mvnrepository.com/artifact/org.scala-lang/scala-library/2.11.11
https://mvnrepository.com/artifact/org.apache.spark/spark-core_2.11/2.2.1
https://mvnrepository.com/artifact/org.apache.spark/spark-sql_2.11/2.2.1
https://mvnrepository.com/artifact/org.apache.spark/spark-streaming_2.11/2.2.1
  
  
  <!-- https://mvnrepository.com/artifact/org.scala-lang/scala-library -->
<dependency>
    <groupId>org.scala-lang</groupId>
    <artifactId>scala-library</artifactId>
    <version>2.11.11</version>
</dependency>

<!-- https://mvnrepository.com/artifact/org.apache.spark/spark-core -->
<dependency>
    <groupId>org.apache.spark</groupId>
    <artifactId>spark-core_2.11</artifactId>
    <version>2.2.1</version>
</dependency>

     <!-- https://mvnrepository.com/artifact/org.apache.spark/spark-sql -->
<dependency>
    <groupId>org.apache.spark</groupId>
    <artifactId>spark-sql_2.11</artifactId>
    <version>2.2.1</version>
</dependency>
    
    <!-- https://mvnrepository.com/artifact/org.apache.spark/spark-streaming -->
<dependency>
    <groupId>org.apache.spark</groupId>
    <artifactId>spark-streaming_2.11</artifactId>
    <version>2.2.1</version>
    <scope>provided</scope>
</dependency>
 
  
  Scala Maven plugin::
  
  http://scala-ide.org/docs/tutorials/m2eclipse/
    
    Help Install new software::
    Maven Scala - http://alchim31.free.fr/m2e-scala/update-site


In [0]:
spark-submit  \
--master yarn \
--deploy-mode cluster \
--executor-memory 1g \
--driver-memory 1g \
--name SparkDEMOCLIENTMODE \
--class com.bdt.SparkDemo \
/home/dorababugjntup/scala-code/SparkDemo.jar /home/dorababugjntup/emp.txt

 
  
  
  
  

In [0]:
stepno,Cache,Broadcat,temptable?,temptable_name,sqlfilename
1,Y,N,Y,emp_temp,folder/step1.hql   -> select * from 
2,N,Y,,,folder/step2.hql -> insert ovewrite tbale   from ... emp_temp



scala code first read the csv file

read line by line
open file step1.hql

if(temptable?)
val df=spark.sql(step1.hql)
 if broadcast??
	sc.boradcast(df)
 if cache?
    df.cache()
df.createOrReplacevieww(emp_temp)
else 
 spark.sql(step2.hql)
 ------------------
 jarfile   csvfilename   folder 
 
 folder->
 csvfile
 step1.hql
 step2.hql
 

In [0]:
FLUME INSTALLATION - 1 Nov 2018

export FLUME_HOME=$HOME/YARN/flume
export FLUME_CONF_DIR=$FLUME_HOME/conf
export FLUME_CLASSPATH=$FLUME_CONF_DIR
export PATH=$PATH:$FLUME_HOME/bin

  
  wget link to software
  tar -xvf apache-flume-1.8.0-bin.tar.gz
 mv apache-flume-1.8.0-bin YARN/flume

vi .bashrc
souce .bashrc
which flume-ng


echo 111111111 >> server.log
tail -f server.log


vi exec_agent.conf

flmagent.sources=exec-source
flmagent.sinks=hdfs-sink
flmagent.channels=ch1


flmagent.sources.exec-source.type = exec
flmagent.sources.exec-source.command = tail -F server.log
flmagent.sources.exec-source.channels = ch1



flmagent.sinks.hdfs-sink.type=hdfs
flmagent.sinks.hdfs-sink.hdfs.path= hdfs://localhost:9000/flumeprog/execflume/
flmagent.sinks.hdfs-sink.hdfs.filePrefix=apacheaccess
flmagent.sinks.hdfs-sink.hdfs.rollInterval=10
flmagent.sinks.hdfs-sink.hdfs.rollSize=0
flmagent.sinks.hdfs-sink.hdfs.writeFormat=Text
flmagent.sinks.hdfs-sink.channel=ch1

flmagent.channels.ch1.type=memory
flmagent.channels.ch1.capacity=1000



--------------
flume-ng agent --conf $FLUME_HOME/conf \
--name flmagent --conf-file $HOME/exec_agent.conf \
-Dflume.root.logger=DEBUG,console


 hdfs dfs  -text /flumeprog/execflume/*



In [0]:
https://github.com/dorababugjntup/SET2
https://github.com/datamitra/scala-code

In [0]:
PIG and PIG LATIN - 2Nov 2018
Download pig::
wget https://www-eu.apache.org/dist/pig/latest/pig-0.17.0.tar.gz
  tar -xvf pig-0.17.0.tar.gz
  mv pig-0.17.0 YARN/pig
  
  vi .bashrc
  export PIG_HOME=$HOME/YARN/pig
  export PATH=$PATH:$PIG_HOME/bin
  
  class code
  wget https://github.com/dorababugjntup/SET2/raw/master/Pig.tar.gz
  
  cd $HOME/Pig/DataSet-Pig

    
    pig -x local
    pig 
    
    
    
    
    divs = LOAD 'NYSE_dividends' ; 

    grunt>     divs = LOAD 'NYSE_dividends'  using PigStorage('\t')
    store divs into 'out' using PigStorage('|')
describe divs;
     divs1= foreach divs generate $1,$2;
  
  grunt>     divs = LOAD 'NYSE_dividends'  using PigStorage('\t') as (f1,f2,f3,f4,f5,f6);

  divs: {f1: bytearray,f2: bytearray,f3: bytearray,f4: bytearray,f5: bytearray,f6: bytearray}
grunt> divs1= foreach divs generate $2,f1,$1;
grunt> divs1= foreach divs generate $2,LOWER(f1) as lw,$1;
divs1: {f3: bytearray,lw: chararray,f2: bytearray}

divs = LOAD 'NYSE_dividends' using PigStorage('\t') as (exchange:chararray, symbol:chararray, date:chararray, dividends:float) ;
divs1= foreach divs generate .. $2;
 divs1 = foreach divs generate UPPER(symbol) as symbol,$2 ;
divs_cyn = filter divs by symbol=='CYN';


every company how much total dividend given? 
group and apply sum , avg


grunt> grpd = group divs by symbol;
grunt> describe grpd;
grpd: {group: chararray,divs: {(exchange: chararray,symbol: chararray,date: chararray,dividends: float)}}
grunt> divs_sum= foreach grpd generate group ,SUM(divs.dividends) as tot_div;
grunt> describe divs_sum;
divs_sum: {group: chararray,tot_div: double}

  
  daily = load 'NYSE_daily' as (exchange, symbol, date, open, high, low, close, volume, adj_close);
  jnd = join divs by (symbol,date) , daily by (symbol,date)
jnd: {divs::exchange: chararray,divs::symbol: chararray,divs::date: chararray,divs::dividends: float,
      daily::exchange: bytearray,daily::symbol: bytearray,daily::date: bytearray,
      daily::open: bytearray,daily::high: bytearray,daily::low: bytearray,
      daily::close: bytearray,daily::volume: bytearray,daily::adj_close: bytearray}

  res = foreach jnd generate divs::exchange as exchange, divs::symbol as symbol,divs::date as date,dividends,open,volume,high;
  res: {divs::exchange: chararray,divs::symbol: chararray,divs::date: chararray,divs::dividends: float,daily::open: bytearray,daily::volume: bytearray,daily::high: bytearray}
  res: {exchange: chararray,symbol: chararray,date: chararray,divs::dividends: float,daily::open: bytearray,daily::volume: bytearray,daily::high: bytearray}

   res_cyn= filter res by symbol=='CYN';
  
  
  grunt > exec file.pig  -> commands doesnt come in history, also aliases doesnt come in memory
  grunt> run file.pig -> commnds and aliases in the file will be available.
  
  
  
  
  23   Z = cogroup A BY $0, B BY $0;
24   Z = cogroup A BY $0 inner, B BY $0;
25   Z = cogroup A BY $0 inner, B BY $0 inner;
20   A = LOAD 'data1' AS (a1:int,a2:int,a3:int);
21   B = LOAD 'data2' AS (b1:int,b2:int);
22   Z = JOIN A BY $0, B BY $0;


In [0]:
HBASE      6-Nov-2018

NOTE::: Download HBASE 1.x version , 2.x may have conflict of ports with GCP Linux
wget https://www-us.apache.org/dist/hbase/2.1.1/hbase-2.1.1-bin.tar.gz
tar -xvf hbase-2.1.1-bin.tar.gz
mv hbase-2.1.1 YARN/hbase 
vi .bashrc
export HBASE_HOME=$HOME/YARN/hbase
export PATH=$PATH:$HBASE_HOME/bin
source .bashrc
cd $HOME/YARN/hbase/conf
echo "export JAVA_HOME=$JAVA_HOME" >>  hbase-env.sh
mkdir $HBASE_HOME/data

vi hbase-site.xml     keep this inside <configuration>

<configuration> 
  <property><name>hbase.tmp.dir</name><value>file://${user.home}/YARN/hbase/data</value></property> 
<!-- <property> <name>hbase.zookeeper.property.clientPort</name><value>2182</value></property> 
 <property><name>hbase.rootdir</name><value>file://${user.home}/YARN/hbase/hbaseroot</value></property>
 <property><name>hbase.zookeeper.property.dataDir</name><value>file://${user.home}/YARN/hbase//zookeeper</value>  </property>    
  -->    
  </configuration>
 
  
  start-hbase.sh     ;;    jps ;;   hbase shell
  
  
  
create 'test','data'
list
scan 'test'
 put 'test', 'r1', 'data:col1', 'val1'
 put 'test', 'r2', 'data:col2', 'val2'
 put 'test', 'r3', 'data:col3', 'val3'


hbase(main):019:0> scan 'test'
ROW                          COLUMN+CELL
 r1                          column=data:col1, timestamp=1541479725362, value=ZZZ
 r2                          column=data:col2, timestamp=1541479677841, value=val2
 r3                          column=data:col3, timestamp=1541479693149, value=val3
3 row(s)
Took 0.0100 seconds
hbase(main):020:0> get 'test','r1','data:col1'
COLUMN                       CELL
 data:col1                   timestamp=1541479725362, value=ZZZ
1 row(s)
Took 0.0447 seconds
  hbase> t.get 'r1', {COLUMN => 'c1', TIMESTAMP => ts1}
get 'test','r1',{ COLUMN =>'data:col1', TIMESTAMP => 1541479615434}


In [0]:
PROJECT::: PART 1 - Data ingestion
      wget https://github.com/datamitra/notes/raw/master/project/data/mysqlsampledatabase.sql
      insert data into mysql
      
(all code editing in windows , migrate the code into GCP using GITHUB/BITBUCKET)
      
Data base names and table naming :::

mysql - tables
mysql control table -> CNTRL - job_run_status
(sqoop_table_name,status(P/E/R/C),extracted_date,max_modified_date_mysql_table)

Hive - incremental database - stg_datalake
Hive - main table database - hdp_datalake
The data type is incremental last modified (Mutable Data)
------------------
Perform incremental logic using hql , Spark-sql
-------------------------

For one of the table we need to get data in real time.
into hdfs folder.
source table -> javap->kafka->flume -> hdfs 
flume use spark dstreams/spark structured streaming
(use spark-submit)

Use Spooling directory and spark-streaming 
to load csv file into hdfs table
--------------------

In [0]:
GIT SETUP
login to github /bitbucket
create repository - Project_June18_WD_Morning

In Windows - install git
Create one folder , run the below command
git clone https://github.com/datamitra/Project_June18_WD_Morning.git
  
In GCP - in Home direcory 
git clone https://github.com/datamitra/Project_June18_WD_Morning.git
  
Test:: create one file in windows, get the data in GCP

 in Windows -
git add *     #add only changed files in production
git commi -m "some message "
git push origin master
git config user.email "datamitra1@gmail.com"
git config user.name "datamitra1"
 
GCP: go inside the project folder
  git pull


In [0]:
Load Source data into Mysql
wget https://github.com/datamitra/Project_June18_WD_Morning/raw/master/data/mysqlsampledatabase.zip
unzip mysqlsampledatabase.zip

mysql -uroot -proot
mysql>show databases;
mysql> source mysqlsampledatabase.sql
show databases;
use classicmodels;
show tables;
describe each table.
mysql> show tables;
+-------------------------+
| Tables_in_classicmodels |
+-------------------------+
| customers               |
| employees               |
| offices                 |
| orderdetails            |
| orders                  |
| payments                |
| productlines            |
| products                |
+-------------------------+
8 rows in set (0.00 sec)



In [0]:
1) Create hive tbale structure and perform full refresh once
2) leave lob(binary lob and clob) columns - for this you need to select few columns alone
(create tables in incr database and main database)

3) make the script reusable
Once you create hive table structure
write import sqoop for incremental last modified
while peforming incr refresh - update contol table

In [0]:
Real time streaming
table -> java producer( use the control for max last modifed) 
kafka -> flue/spark streaming into hive table.
